# Importando bibliotecas

In [ ]:
!pip install -U -q PyDrive

In [ ]:
# importando as bibliotecas necessárias para o bom funcionamento do código
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# Carregando dados

In [ ]:
# Autenticação para acesso aos dados
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Download dos dados para esta máquina virtual
downloaded = drive.CreateFile({'id': '1DUV-TOm5NhiaDfBgewwh26Ky7jgoC1mU'})
downloaded.GetContentFile('data.csv')

In [ ]:
# Carregando os dados baixados no dataframe 
df = pd.read_csv('data.csv')
df.head()

# df = df[(~df.ind_atritado.isnull()) | (~df.ind_engajado.isnull()) | (~df.ind_novo_cliente.isnull())]
df.head()

,anomes,num_cpf_hash,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,vlr_renda,cod_rating,ind_atrito,ind_engaj,ind_novo_cli
0,202104,fffff8b0db8eff291be8b83f8885f52c52782bb42c3c4b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,202104,ffffd54b45ec46113523184fc07185a0d5cbfa876a07ba...,35943.74,5815.5,NaN,377.0,1.0,NaN,24.0,NaN,10.0,NaN,A,NaN,1.0,NaN
2,202104,ffffd47a92b3e4291c013033ae528708a19eaede50f78e...,6288.22,NaN,NaN,257.0,NaN,NaN,22.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN
3,202104,ffffc102ddd37ec29e985a4564e85a2bace79a85ebff5e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
4,202104,ffffbd4a3d42a12e07b1202d68c33d43220c42c8a55160...,1238.93,1400.0,NaN,773.0,2.0,NaN,21.0,NaN,NaN,NaN,A,NaN,1.0,NaN


# **Limpeza dos dados - parte 1**

* Em primeiro lugar, é necessário retirar aquelas linhas que não possuem cod_rating (parâmetro utilizado internamente pelo banco para classificar seus clientes) a fim de facilitar o aprendizado do nosso modelo preditivo, visto que, se não possui cod_rating, necessariamente a linha não pertence a um cliente do banco.

* Após a resolução desse primeiro empecilho, trocamos todos os valores nulos das colunas "num_atend", "num_atend_atrs", "qtd_reclm", "qtd_restr", "ind_atrito", "ind_engaj", "ind_novo_client" e "vlr_score" para o valor zero (0) porque o modelo preditivo não consegue identificar e nem manipular dados diferentes de números.

* Finalizando essa primeira parte, mudamos os parâmetros do cod_rating de letras (A, B, C, D, E, F, G, H e HH) para números (0, 1, 2, 3, 4, 5, 6, 7 e 8, respectivamente)


In [ ]:
print(len(df))
df = df[~df.cod_rating.isnull()]
print(len(df))
df[['num_atend', 'num_atend_atrs', 'qtd_reclm', 'qtd_restr', 'ind_atrito', 'ind_engaj', 'ind_novo_cli', 'vlr_score', 'num_produtos']] = df[['num_atend','num_atend_atrs','qtd_reclm', 'qtd_restr', 'ind_atrito', 'ind_engaj', 'ind_novo_cli', 'vlr_score', 'num_produtos']].fillna(0)

le = LabelEncoder()
df['cod_rating'] = le.fit_transform(df['cod_rating'])

df

12032493
6186773


,anomes,num_cpf_hash,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,vlr_renda,cod_rating,ind_atrito,ind_engaj,ind_novo_cli
1,202104,ffffd54b45ec46113523184fc07185a0d5cbfa876a07ba...,35943.74,5815.50,0.0,377.0,1.0,0.0,24.0,0.0,10.0,NaN,0,0.0,1.0,0.0
4,202104,ffffbd4a3d42a12e07b1202d68c33d43220c42c8a55160...,1238.93,1400.00,0.0,773.0,2.0,0.0,21.0,0.0,0.0,NaN,0,0.0,1.0,0.0
7,202104,ffff6070251f9e50f54190b914ed6fd735e11064d661ab...,12607.75,31.70,0.0,0.0,0.0,0.0,9.0,0.0,0.0,NaN,9,0.0,0.0,0.0
8,202104,ffff4ef886c28af8029c6b7d504942e6c9ef13021e9a35...,81506.54,5243.45,0.0,560.0,1.0,0.0,14.0,0.0,0.0,NaN,0,0.0,0.0,0.0
9,202104,ffff4bbb4074d7894174fd94c685edb3fd7fc9b7b1e3c1...,112918.62,7252.64,0.0,374.0,2.0,0.0,23.0,0.0,0.0,NaN,0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12032484,202204,00008503efab22c4b17633da58b19557d135887e01db0e...,36333.58,5942.14,0.0,538.0,4.0,0.0,40.0,0.0,1.0,NaN,2,0.0,1.0,0.0
12032485,202204,00008166c6d8213d5372a4a6336322f14e7f15b982cf5d...,43054.69,2620.89,0.0,437.0,2.0,0.0,17.0,0.0,8.0,NaN,0,0.0,0.0,0.0
12032486,202204,00006b833ab2b4c8220f61fdd90c5833040030ff1b015e...,NaN,11239.24,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,9,0.0,0.0,0.0
12032487,202204,0000567d0d7f3813befdc85b65838d685ec6d89bb067bd...,21959.05,4017.00,0.0,410.0,1.0,0.0,15.0,0.0,4.0,NaN,8,0.0,0.0,0.0


# **Limpeza dos dados - parte 2**

 #### CPF único
* A fim de que não haja mais de um cpf por linha na nossa base de dados, decidimos escolher, por ora, uma safra aleatória. A escolha aleatória foi feita para que não exista um viés enquanto selecionamos uma safra específica, podendo ter mais ou menos clientes atritados por período. 


In [ ]:
# importando as bibliotecas que serão utilizadas 
import plotly.express as px
import random

In [ ]:
# geração de um número aleatório referente à safra
random_number = random.randint(1,12)
# eliminação de safras duplicadas, restando apenas 12 safras, referentes aos meses do ano
safra_unica= df.drop_duplicates('anomes')
# escolha de uma safra aleatória entre as 12 existentes
safra_escolhida = safra_unica['anomes'].iloc[random_number]
# seleção dos cpfs que se enquadram na safra escolhida, existindo um cpf por linha, apenas
todos_cpfs_safra = df[df['anomes']== safra_escolhida]

In [ ]:
# Atribuição, ao data frame antigo, do novo data frame com cpf's únicos 
df = todos_cpfs_safra

# **Limpeza dos dados - parte 3**

* Nessa etapa, retiramos aquelas linhas (que representam pessoas) que possuem valor de crédito e saldo como **nulos** devido ao fato de que há uma alta probabilidade de que essas possoas não possuam conta no Banco Pan. 

* Posteriormente, a partir de uma verificação, percebemos que existem linhas que, apesar de possuirem valores concretos de score, valor de saldo, crédito, número de atendimentos, etc. não possuem informações na coluna **renda**. Dessa forma, a fim de potencializar o funcionamento do modelo preditivo nós, após perceber a irrelevância dessa coluna para a construção desse, decidimos retirá-la da tabela. Além dela, retiramos, também, a coluna que continha o cpf, visto que, por ter sofrido um processo de hash, possuia letras em sua composição e não era compreendido pela predição. 



In [ ]:
# retirada de todos os valores de crédito e saldo que estão nulos
df = df[~df.vlr_credito.isnull() & ~df.vlr_saldo.isnull()]
# exclusão das colunas renda e cpf
df = df.drop('vlr_renda', axis=1)
df = df.drop('num_cpf_hash', axis=1)
# verificação das mudanças realizadas:
df

,anomes,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,cod_rating,ind_atrito,ind_engaj,ind_novo_cli
8421938,202201,34455.60,5683.73,0.0,438.0,2.0,0.0,12.0,0.0,2.0,0,0.0,1.0,0.0
8421941,202201,4086.83,1400.32,0.0,777.0,2.0,0.0,32.0,0.0,0.0,0,0.0,1.0,0.0
8421945,202201,75630.94,5219.01,0.0,516.0,1.0,0.0,14.0,0.0,0.0,0,0.0,1.0,0.0
8421946,202201,118765.46,6535.31,0.0,373.0,2.0,0.0,22.0,0.0,1.0,0,0.0,1.0,0.0
8421951,202201,9897.73,1752.06,0.0,516.0,1.0,0.0,9.0,0.0,0.0,0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9300576,202201,2446.02,688.85,0.0,731.0,4.0,0.0,11.0,0.0,0.0,0,0.0,1.0,0.0
9300577,202201,35662.30,5518.16,0.0,913.0,5.0,0.0,45.0,0.0,0.0,0,0.0,1.0,0.0
9300578,202201,40176.27,2676.17,0.0,389.0,3.0,0.0,13.0,0.0,8.0,0,0.0,0.0,0.0
9300581,202201,34454.97,4017.00,0.0,403.0,1.0,0.0,14.0,0.0,4.0,8,0.0,0.0,0.0


* Agora, com os nulos tratados, queremos conferir se existe algum dado no data frame que permaneça nulo, a fim de corrirgir caso o resultado seja diferente de zero. 

In [ ]:
# conferindo a existência de dados nulos
df.isnull().sum()

anomes            0
vlr_credito       0
vlr_saldo         0
num_atend_atrs    0
vlr_score         0
num_produtos      0
num_atend         0
qtd_oper          0
qtd_reclm         0
qtd_restr         0
cod_rating        0
ind_atrito        0
ind_engaj         0
ind_novo_cli      0
dtype: int64

* Abaixo, temos a criação de um protótipo de modelo que foi criado para a testagem dos métodos de limpeza pré existentes. Ele não está sendo criado para a utilização, estando no notebook apenas para fins de teste.


In [ ]:
# Dividindo x e y
x = df.drop(columns = ['ind_atrito','ind_engaj', 'ind_novo_cli']).values
y = df[['ind_atrito','ind_engaj', 'ind_novo_cli']].values

print(x)
print(y)

# Dividindo dados para treino e dados para teste
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.3, 
                                                    random_state = 42)

# Treinando o modelo
print(x_test)
print(y_test)

# Treinando o modelo
model = LinearRegression().fit(x_train, y_train)

print(x_test)

# Fazendo as predições
y_pred = model.predict(x_test)
print(y_pred)

# Calculando o erro r2- medida de qualidade do modelo
r2_score(y_test, y_pred)

[[2.022010e+05 3.445560e+04 5.683730e+03 ... 0.000000e+00 2.000000e+00
  0.000000e+00]
 [2.022010e+05 4.086830e+03 1.400320e+03 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [2.022010e+05 7.563094e+04 5.219010e+03 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 ...
 [2.022010e+05 4.017627e+04 2.676170e+03 ... 0.000000e+00 8.000000e+00
  0.000000e+00]
 [2.022010e+05 3.445497e+04 4.017000e+03 ... 0.000000e+00 4.000000e+00
  8.000000e+00]
 [2.022010e+05 3.063052e+04 1.695740e+03 ... 0.000000e+00 6.000000e+00
  0.000000e+00]]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
[[2.022010e+05 5.372260e+03 1.480000e+03 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [2.022010e+05 2.610644e+04 6.010400e+02 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [2.022010e+05 3.155970e+03 5.019000e+01 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 ...
 [2.022010e+05 2.216240e+03 1.918300e+02 ... 0.000000e+00 2.000000e+00
  0.000000e+00]
 [2.022010e+05 3.880414e+04 4.017340e+03 

0.43625607388842963

* A seguir, estamos calculando algumas médias que podem ser relevantes para o posterior treinamento do nosso modelo preditivo.


In [ ]:
#Calculando Médias

medias = df[['vlr_credito', 'vlr_saldo', 'vlr_score', 'num_atend_atrs', 'num_produtos']].sum()/len(df)
print(medias)
medias.dtypes

medias = pd.DataFrame(medias)

fig = px.bar(medias, y=medias[0], height=400, width=600)
fig.update_xaxes(type='category')
fig.show()

vlr_credito       37730.569733
vlr_saldo          5639.187711
vlr_score           524.751478
num_atend_atrs        0.000894
num_produtos          1.794347
dtype: float64


* Aqui, estamos comparando a quantidade de atendimentos existentes com a quantidade de atendimentos que se encontram atrasados.


In [ ]:
# Média de atendimentos e atendimentos atrasados:
atends = df[['num_atend', 'num_atend_atrs']].sum()/len(df)
df_atends = pd.DataFrame(atends)

# geração de um gráfico com as médias obtidas anteriormente
fig = px.bar(df_atends, y=df_atends[0], height=400, width=600)
fig.update_xaxes(type='category')
fig.show()

In [ ]:
# descrição da quantidade de reclamações
num_reclm = df['qtd_reclm'].describe()
# criação de um data frame com os números encontrados anteriormente.
num_reclm_1 = pd.DataFrame(num_reclm)
# retirada do parâmtro count desse df (data frame) criado
num_reclm_1 = num_reclm_1.drop('count')

# criação de um gráfico para a melhor visualização desses valores
figReclm = px.bar(num_reclm_1, y=num_reclm_1['qtd_reclm'], height=400, width=600)
figReclm.update_xaxes(type='category')
figReclm.show()

In [ ]:
#Análise de clientes que buscam novos produtos/atritados

#NAN das colunas vlr_credito, vlr_saldo e num_produtos sendo trocados por 0;
data= df[['vlr_credito', 'vlr_saldo', 'num_produtos']].replace(np.nan, 0)

#Gera-se um novo dataframe com as informações específicas
data = pd.DataFrame(data);

#Pega-se as pessoas que possuem um saldo bem maior do que o crédito e que possuem produtos no banco.
data[(data.vlr_credito < 1500) & (data.vlr_saldo > data.vlr_credito) & (data.vlr_saldo > 10000) & (data.num_produtos >= 1) & (data.vlr_credito > 0)]

,vlr_credito,vlr_saldo,num_produtos
8432104,1059.27,11149.27,3.0
8434335,1402.64,11659.84,2.0
8436619,1422.97,14785.00,1.0
8437498,294.12,13149.12,1.0
8438083,1222.82,17060.92,2.0
...,...,...,...
9292847,326.18,28859.89,2.0
9293394,1166.46,16424.46,1.0
9294045,959.27,12199.66,1.0
9298098,626.18,22816.94,1.0


In [ ]:
#Análise de cliente a partir do score e do cod_rating

#NAN das colunas cod_rating, vlr_score e num_produtos sendo trocados por 0;
data_score = df[['cod_rating', 'vlr_score', 'num_produtos']].replace(np.nan, 0);

#Gera-se um novo dataframe com as informações específicas
data_score = pd.DataFrame(data_score);

#Selecionando pessoas que têm produtos, possuem score maior do que 400, e que não estejam com rating E, F ou G
print("Quantidade de pessoas com métricas normais: ")
print(len(data_score[(data_score.vlr_score > 400) & (data_score.vlr_score != 0) & ((data_score.cod_rating != 'E') | (data_score.cod_rating != 'F') | (data_score.cod_rating != 'G') | (data_score.cod_rating != 0)) & (data_score.num_produtos >= 1)]))

#Selecionando pessoas que têm produtos, possuem score menor do que 400, e que estejam com rating E, F ou G
print("Quantidade de pessoas com métricas anormais: ")
len(data_score[(data_score.vlr_score < 400) & (data_score.vlr_score != 0) & ((data_score.cod_rating == 'E') | (data_score.cod_rating == 'F') | (data_score.cod_rating == 'G')) & (data_score.num_produtos >= 1)])

Quantidade de pessoas com métricas normais: 
267436
Quantidade de pessoas com métricas anormais: 


0

In [ ]:
#NAN das colunas num_atend, num_atend_atrs sendo trocados por 0;
data = df[['num_atend', 'num_atend_atrs']].replace(np.nan, 0);

#Gera-se um novo dataframe com as informações específicas
data = pd.DataFrame(data)

#Pega-se a quantidade de atendimentos que foram feitos e estão em dia.
length_nao_atrs = len(data[(data.num_atend > 0) & (data.num_atend_atrs == 0)])

print('Quantidade de atendimentos que foram feitos mas não estão em atraso: ', length_nao_atrs)

#Pega-se a quantidade de atendimentos que foram feitos e não estão em dia.
length_atrs = len(data[(data.num_atend > 0) & (data.num_atend_atrs > 0)])

print('========================================================================')

print('Quantidade de atendimentos que foram feitos e estão em atraso: ', length_atrs)

Quantidade de atendimentos que foram feitos mas não estão em atraso:  1213
Quantidade de atendimentos que foram feitos e estão em atraso:  313


In [ ]:
df

,anomes,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,cod_rating,ind_atrito,ind_engaj,ind_novo_cli
8421938,202201,34455.60,5683.73,0.0,438.0,2.0,0.0,12.0,0.0,2.0,0,0.0,1.0,0.0
8421941,202201,4086.83,1400.32,0.0,777.0,2.0,0.0,32.0,0.0,0.0,0,0.0,1.0,0.0
8421945,202201,75630.94,5219.01,0.0,516.0,1.0,0.0,14.0,0.0,0.0,0,0.0,1.0,0.0
8421946,202201,118765.46,6535.31,0.0,373.0,2.0,0.0,22.0,0.0,1.0,0,0.0,1.0,0.0
8421951,202201,9897.73,1752.06,0.0,516.0,1.0,0.0,9.0,0.0,0.0,0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9300576,202201,2446.02,688.85,0.0,731.0,4.0,0.0,11.0,0.0,0.0,0,0.0,1.0,0.0
9300577,202201,35662.30,5518.16,0.0,913.0,5.0,0.0,45.0,0.0,0.0,0,0.0,1.0,0.0
9300578,202201,40176.27,2676.17,0.0,389.0,3.0,0.0,13.0,0.0,8.0,0,0.0,0.0,0.0
9300581,202201,34454.97,4017.00,0.0,403.0,1.0,0.0,14.0,0.0,4.0,8,0.0,0.0,0.0


# Seleção de dados
 
**Após a de limpeza dos dados ausentes e dos dados nulos, definimos quais as features que serão utilizadas na construção do modelo preditivo. Para fazer tal escolha, consideramos os atributos da base de dados do cliente que são relevantes no processo de análise da conduta de um cliente do Banco PAN.**
 
• Safra (“anomes”) --> Ao analisar a safra (ano e mês), conseguimos observar o comportamento do cliente ao longo do tempo e estimar seu comportamento futuro enquanto cliente do banco.
 
• Valor de crédito (“vlr_credito”) --> O valor de crédito no mercado avalia a situação do cliente no mercado ao declarar seu valor total de crédito,sendo indicador de possíveis atritos entre cliente e banco, bem como indicações de novos clientes ou clientes que possam ter a intenção de adquirir novos produtos.
 
• Valor de saldo (“vlr_saldo”) --> O valor total de saldo do cliente indica se há atrito na relação que será predita pelo modelo artificial. 
    
• Número de atendimentos (“num_atend” e “num_atend_atrs”) --> O número total de atendimentos e número de atendimentos atrasados é um dos principais indicativos de atrito de um cliente com o Banco Pan.
    
• Valor do score (“vlr_score”)  --> O valor do score no Serasa de um cliente é capaz de identificar a necessidade do cliente de adquirir, por exemplo, certo valor de crédito no banco.
 
• Número de produtos (“num_produtos”) --> Quanto maior o número de serviços aderidos, menor pode ser o nível de atrito com a instituição, considerando a preferência pelo Banco PAN quanto a esses serviços.
 
• Quantidade de operações (“qtd_oper”) --> Quando um cliente possui muitas operações referentes a um mesmo serviço, deve ser considerada a preferência pelo Banco PAN frente a outras instituições financeiras.
 
• Quantidade de reclamações (“qtd_reclm”) --> Se um cliente possui muitas reclamações, ele se sente insatisfeito com os serviços oferecidos pelo Banco PAN.
 
• Quantidade de restritivos no mercado (“qtd_restr”) --> Considera as restrições financeiras do cliente em outras instituições financeiras.

• Rating do cliente (“cod_rating”) --> Refere-se ao risco que o cliente representa para o banco, indicando se existem atritos nessa relação ou se é provável que o cliente adquira novos produtos na instituição.
 
• Índice de atrito (“ind_atritado”) --> É uma métrica interna do banco que quantifica possíveis conflitos entre o Banco PAN e seus clientes. Se esse índice for alto, pode indicar que o cliente está ligando para efetuar uma reclamação.
 
• Índice de engajamento (“ind_engajado”) --> Reflete o quão engajado o cliente é com a instituição, auxiliando o modelo preditivo a analisar se é alta ou baixa a probabilidade de que o cliente contate o banco para adquirir novos produtos ou serviços.
 
• Índice de identificação de novo cliente (“ind_novo_cliente”) --> Identifica potenciais clientes do Banco PAN, os quais serão identificados ao entrarem em contato com o atendimento da instituição, de modo que a intenção de abrir uma conta no banco seja predita pelo modelo preditivo.
 
**Por fim, foram definidos os campos que não serão utilizados na construção da lógica do modelo preditivo, os quais encontram-se dispostos abaixo.**
 
• Número do CPF (“num_cpf”) --> Os clientes não serão avaliados individualmente, o modelo preditivo irá considerar um conjunto de clientes. Além disso, os clientes devem ser mantidos anônimos e os valores deste campo são strings, não sendo reconhecidos pelo modelo. 
 
• Valor da renda (“vlr_renda") --> A maioria dos valores para esse campo são nulos (NaN), não havendo uma quantidade suficiente de registros para que o atributo em questão influencie as predições realizadas.